In [1]:
# Exercises with Coordination and Management

In [2]:
# Create Spark Context with SparkConf
from pyspark import SparkConf, SparkContext
conf = SparkConf()
sc = SparkContext.getOrCreate(conf)

25/02/01 13:47:49 WARN Utils: Your hostname, student-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/02/01 13:47:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/01 13:47:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Exercise 1
input_data = [("Simón","Bolivar","VEN","489 895 965"),
    ("Fidel","Castro","CU","956 268 348"),
    ("Jose","Doroteo","MEX","985 621 444"),
    ("Ernesto","Guevara","AR","895 325 481"),
    ("Hugo","Chávez","VE","489 895 965"),
    ("Camilo","Cienfuegos","CUB","956 268 348"),
    ("Andrés","López","MX","985 621 444"),
    ("Juan Domingo","Perón","ARG","985 621 444"),
  ]

country_codes = {"VEN": "+58", "CU": "+53", "MEX": "+52", "AR": "+54", "VE": "+58", "CUB": "+53", "MX": "+52", "ARG": "+54"}

broadcast_codes = sc.broadcast(country_codes)

rdd = sc.parallelize(input_data)

update_rdd = rdd.map(lambda x: (x[0], x[1], x[2], broadcast_codes.value.get(x[2], "") + " " + x[3]))

result = update_rdd.collect()
for item in result: 
  print(item)


('Simón', 'Bolivar', 'VEN', '+58 489 895 965')
('Fidel', 'Castro', 'CU', '+53 956 268 348')
('Jose', 'Doroteo', 'MEX', '+52 985 621 444')
('Ernesto', 'Guevara', 'AR', '+54 895 325 481')
('Hugo', 'Chávez', 'VE', '+58 489 895 965')
('Camilo', 'Cienfuegos', 'CUB', '+53 956 268 348')
('Andrés', 'López', 'MX', '+52 985 621 444')
('Juan Domingo', 'Perón', 'ARG', '+54 985 621 444')


In [4]:
# Exercise 2
input_file_path="bible.txt"
bible_rdd = sc.textFile(input_file_path)

line_with_to = bible_rdd.filter(lambda line: "to" in line).count()
total_lines = bible_rdd.count()

print("Lines containing 'to': ", line_with_to)
print("Total lines in bible.txt: ", total_lines)

Lines containing 'to':  16548
Total lines in bible.txt:  30383


In [6]:
# Exercise 3
input_file_path="pagecounts-20100806-030000"

pagecounts_rdd = sc.textFile(input_file_path)
pagecounts_rdd.saveAsTextFile("output/original")

pagecounts_rdd.repartition(2).saveAsTextFile("output/two_partitions")

pagecounts_rdd.coalesce(1).saveAsTextFile("output/single_file")

In [7]:
# Exercise 4
import time

input_file_path="pagecounts-20100806-030000"
pagecounts_rdd = sc.textFile(input_file_path)
start_time = time.time()
non_cached_count = pagecounts_rdd.count()
non_cached_time = time.time() - start_time

cached_rdd = pagecounts_rdd.cache()
start_time = time.time()
cached_count = cached_rdd.count()
cached_time = time.time() - start_time

print("Non-cached time: ", non_cached_time, " seconds")
print("Cached time: ", cached_time, " seconds")

Non-cached time:  3.4314632415771484  seconds
Cached time:  8.373480558395386  seconds


In [8]:
sc.stop()